In [4]:
import pandas as pd
import numpy as np
import pickle

import warnings
from datetime import datetime
from datetime import date,timedelta
from dateutil.relativedelta import relativedelta
np.seterr(divide='ignore', invalid='ignore')
warnings.filterwarnings("ignore", category=DeprecationWarning) 
warnings.filterwarnings("ignore")

def index_df_dates(df,date1,date2):
    df = df.set_index(['dates'])
    try:
        df.index = pd.to_datetime(df.index,format='%Y-%m-%d')
    except:
        df.index = pd.to_datetime(df.index)
        
    df['datetime'] = df.index
    #if df.dates.diff(periods=-1).iloc[0] != timedelta(days=-1): #how do I change to if any
    print('interpolating data...')
    upsampled = df.resample('D')
    df = upsampled.interpolate(method='linear')
    df = df.loc[date1:date2]
    return df

In [5]:
btc_daily_df = pd.read_pickle('raw_data/btc_price_daily_all.pkl')
btc_daily_df = btc_daily_df.reindex(index=btc_daily_df.index[::-1])
#social interest
reddit_subscribers_df = pd.read_pickle('raw_data/reddit_subscribers.pkl') #1
tweets_daily_df = pd.read_pickle('raw_data/tweets4years.pkl') #2
google_trend_df = pd.read_csv('raw_data/multiTimeline5yrs.csv', header=1, names=['Week','worldwide_interest']) #3


#technology interest (mining demand, coding, meetups?)
nvidia_stock_df = pd.read_csv('raw_data/HistoricalQuotes.csv') #4
commits_daily_df = pd.read_pickle('raw_data/commitsbtcall.pkl') #5

#adoptability (transacting in btc, number of wallets)
bitcoin_transcations_df = pd.read_pickle('raw_data/transactions_alltime.pkl') #6
bitcoin_transcations_fees_df = pd.read_pickle('raw_data/transactions_fees_alltime.pkl') #7
active_addresses = pd.read_pickle('raw_data/addresses_alltime.pkl') #8

nvidia_stock_df.rename(index=str, columns={"date": "dates"},inplace=True)
nvidia_stock_df.drop(nvidia_stock_df.index[0], inplace=True)
nvidia_stock_df = nvidia_stock_df.reindex(index=nvidia_stock_df.index[::-1])
nvidia_stock_df = nvidia_stock_df[['dates','close']]

google_trend_df.rename(index=str, columns={"Week": "dates"},inplace=True)

list_stuff = [[pd.to_datetime(btc_daily_df.dates).min(),pd.to_datetime(btc_daily_df.dates).max()],\
        [tweets_daily_df.dates.min(),tweets_daily_df.dates.max()],\
        [pd.to_datetime(google_trend_df.dates.min()),pd.to_datetime(google_trend_df.dates.max())],\
        [pd.to_datetime(nvidia_stock_df.dates.min()),pd.to_datetime(nvidia_stock_df.dates.max())],\
        [commits_daily_df.dates.min(),commits_daily_df.dates.max()],\
        [bitcoin_transcations_df.dates.min(),bitcoin_transcations_df.dates.max()],\
        [active_addresses.dates.min(),active_addresses.dates.max()]]

X = np.array(list_stuff)
max_per_col = X.max(axis=0)
max_per_row = google_trend_df.dates.max()

In [6]:
d1 = max_per_col[0]
d2 = max_per_row
names_list = [btc_daily_df, tweets_daily_df, google_trend_df,\
              nvidia_stock_df, commits_daily_df, bitcoin_transcations_df, bitcoin_transcations_fees_df, active_addresses]
new_names_list = []
for name in names_list:
    new_name = index_df_dates(name,d1,d2)
    new_name = new_name.drop('datetime',axis=1)
    new_names_list.append(new_name)
    
    print(len(new_name))
    #new_names_list.append(new_name)

interpolating data...
1552
interpolating data...
1552
interpolating data...
1552
interpolating data...
1552
interpolating data...
1552
interpolating data...
1552
interpolating data...
1552
interpolating data...
1552


In [7]:
new_df = pd.concat(new_names_list, axis=1)
new_df.tail()

,prices,tweets,worldwide_interest,close,commits,transactions,transaction_fees,address
dates,,,,,,,,
2018-07-04,6550.87,40065.0,9.571429,239.785,1.0,206654.0,0.687,530349.0
2018-07-05,6599.71,38128.0,9.428571,242.730,2.0,199463.0,0.654,450145.0
2018-07-06,6638.69,38468.0,9.285714,247.330,1.0,200833.0,0.710,564683.0
2018-07-07,6668.71,32273.0,9.142857,247.970,1.0,170004.0,0.644,394186.0
2018-07-08,6857.80,31005.0,9.000000,248.610,3.0,155527.0,0.680,392740.0


In [8]:
new_df = new_df.dropna(axis=1)

In [9]:
new_df['prices'] = new_df['prices'].astype(float)
all_features = new_df
all_features.to_pickle('raw_data/all_features.pkl')